View [idtitleauthor] : id - 작품 ID / title - 작품 제목 / author - 작품 작가
view "idtitleauthor"에서 가져온 title을 검색
* 검색 시, 띄어쓰기 및 특수문자 허용 X
분석 결과를 id의 mentioned row에 삽입

# 인스타그램 수집 내용 | 포스트 내용, 작성 날짜, 총 수집 개수

In [1]:
import pymysql.cursors
import pandas as pd

conn = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='root',
    passwd='root',
    db='webnoveldb',
    charset='utf8')

curs = conn.cursor()
sql = "select * from idtitleauthor"
curs.execute(sql)

result = curs.fetchall()
print(result)
#df=pd.DataFrame(result, columns=["id", "novel", "author"])
#df

((5, '내 안에 마교있다', '초(류희윤)'), (6, '내 머릿속에 2000년 마법 역사', '산천'), (7, '이세계 천재 화가', '블루피스'), (8, '회귀의 전설', '김광수'), (9, '내가 키운 S급들', '근서'), (10, '재능이 자꾸 늘어', '달의등대'), (11, '아포칼립스의 고인물', '슬리버'), (12, '로그인하자마자 VIP', '토이카'), (13, '패왕이 되는 방법', '홀로선별'), (14, '웹소설만 읽으면 강해져', '그루밤'), (15, '수의사 진태민', '서건주'), (16, '이번엔 진짜 재벌!', '별그림자'), (17, '백작가의 망나니가 되었다', '유려한'), (18, '나 혼자만 레벨업', '추공'), (19, '환생표사', '신갈나무'), (20, '명문고 EX급 조연의 리플레이', '기월월'), (21, '망나니 1왕자가 되었다', '글럼프'), (22, '달콤, 찬란한 재벌기', '남희성'), (23, '리퍼-무공수확자', '곽승범'), (24, 'VVVIP 차원 쇼퍼', '오늘도요'), (25, '용사를 방치했더니 차원 최강', '갯장어'), (26, '회귀해서 다해먹음', '우성'), (27, '상단주의 망나니 아들이 되었습니다', '정선비'), (28, '일신상의 이유로 잠시 휴재합니다', '크래커'), (29, '나만 1회차', '플래터'), (30, '내가 만든 AI가 성좌가 되었다', '똥맛된장'), (31, '번개를 맞았다', '탄탄비'), (32, '히든 퀘스트로 탑셰프', '글망쟁이'), (33, '10서클 직전에 환생', '레드리프'), (34, '무신귀환록', '묘수'), (35, '아기부터 시작하는 연예계 생활', '어흥선생'), (36, '주인공이 불행을 다 막음', '언늘'), (37, '최강재능으로 회귀하다', '신수.'), (38, '나도 정상은 아니더라', 'achime'), (39, '무림에서 레벨업', '오준환'), (40, '질풍패도', 'codezer

In [6]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import os
import urllib.request
import warnings # 경고메세지 제거
import requests
from datetime import datetime, timedelta
import pandas as pd
import re

def cleaned(term):
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', term)
    return text

warnings.filterwarnings(action='ignore') # 경고 메세지 제거

# 로그인 계정 정보 [ID : dawaplease / PWD : dawa20152017]
ig_id = 'dawaplease'
ig_pwd = 'dawa20152017'

# 크롬드라이버 호출
driver = webdriver.Chrome("C:/project/chromedriver.exe")

# 로그인 [ID : dawaplease / PW : dawa20152017]
driver.get('https://www.instagram.com/accounts/login/')

time.sleep(3)

id_input = driver.find_elements_by_css_selector('#loginForm > div > div > div > label > input')[0]

id_input.send_keys(ig_id)
password_input = driver.find_elements_by_css_selector('#loginForm > div > div > div > label > input')[1]
password_input.send_keys(ig_pwd)
password_input.submit()
time.sleep(3)

# 작품 검색
search_key = input("!주의! 검색 명내 띄어쓰기 사용 X !\n작품 검색 #")
search_key=cleaned(search_key).replace(" ","")
print(search_key+" 검색 시작\n")
search_ig = "https://www.instagram.com/explore/tags/" + search_key + "/"

# 작품 검색 페이지 로드
driver.get(search_ig)
time.sleep(3) #웹 페이지 로드 시간 : 3초

# 마지막 결과까지 스크롤
SCROLL_PAUSE_TIME = 1.0
post_list = []

# 중복 제거
def duplicate(my_list):
    my_set = set(my_list) # 집합으로 변환
    new_list = list(my_set) # 리스트로 변환
    return new_list

# 포스트 링크 추출
while True:
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")

    for link1 in soup.find_all(name="div",attrs={"class":"Nnq7C weEfm"}):
        for num in range(3):
            try:
                title = link1.select('a')[num]
                real = title.attrs['href']
                post_list.append(real)
            except:
                break

    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
            
        else:
            last_height = new_height
            continue

post_list = duplicate(post_list) # 중복포스트 제거
post_num = len(post_list) # 검색 결과 개수
print("총 "+str(post_num)+"개의 데이터.\n")

# 포스트 데이터 추출
csvtext = []

for i in range(0, post_num):
    csvtext.append([])
    post_url = 'https://www.instagram.com'+ post_list[i]
    driver.get(post_url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    # 내용
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ''
    csvtext[i].append(content)
    print(content)
    
    # 작성날짜
    try:
        date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    except:
        date = ''
    csvtext[i].append(date)
    print(date)

print(post_num, "개의 데이터 받아오는 중.\n")
total=post_num
print("저장 완료.\n")

driver.close() # 크롬 드라이버 종료

# 총 수집 개수 : total | 수집 내용 csvtext [list 타입]

!주의! 검색 명내 띄어쓰기 사용 X !
작품 검색 #차원 보따리상입니다만
차원보따리상입니다만 검색 시작

총 0개의 데이터.

0 개의 데이터 받아오는 중.

저장 완료.



In [7]:
# DataFrame으로 수집한 결과 확인하기
import pandas as pd

data = pd.DataFrame(csvtext, columns = ['내용', '작성 날짜'])
data

,내용,작성 날짜


# 코드 제작자 : 한승주